# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
from pprint import pprint
import numpy as np

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tura,25.5198,90.2201,28.78,56,38,2.03,IN,1730351944
1,1,longyearbyen,78.2186,15.6401,-7.09,73,75,1.03,SJ,1730351946
2,2,bilibino,68.0546,166.4372,-14.47,91,100,3.31,RU,1730351947
3,3,grytviken,-54.2811,-36.5092,0.06,86,31,4.14,GS,1730351948
4,4,ancud,-41.8697,-73.8203,8.22,95,15,1.76,CL,1730351949


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

##### I dont exactly know how they accomplished this so here is some reference code from geoviews_demo_solution

In [3]:
%%capture --no-display
# change the data type of Lng and Lat to float 
city_data_df["Lat"] = city_data_df["Lat"].astype(float)
city_data_df["Lng"] = city_data_df["Lng"].astype(float)

# Configure the map plot
map_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500,
    size = 'Humidity',
    scale = .5,
    color = 'City'
)

# Display the map
map_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Check to see if there are any missing data points
column_count = city_data_df.count()
column_count

City_ID       574
City          574
Lat           574
Lng           574
Max Temp      574
Humidity      574
Cloudiness    574
Wind Speed    574
Country       570
Date          574
dtype: int64

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
# It seems like there are 4 counties missing from my dataset. Thus, I need to drop the rows with no country
cleaned_city_data = city_data_df.dropna()

# filter out citys that don't fit criteria
# I prefer wether between 48 and 70 degrees
cleaned_city_data = cleaned_city_data[(cleaned_city_data['Max Temp'] >= 8.9) & (cleaned_city_data['Max Temp'] <= 21.2)]

# I am okay with some clouds in the sky
cleaned_city_data = cleaned_city_data[(cleaned_city_data['Cloudiness'] >= 10) & (cleaned_city_data['Cloudiness'] <= 70)]

# I want at least a light breeze, but I dont want wind speeds that are too strong
cleaned_city_data = cleaned_city_data[(cleaned_city_data['Wind Speed'] >= 1.6) & (cleaned_city_data['Wind Speed'] <= 5.8)]

cleaned_city_data.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,port alfred,-33.5906,26.8910,16.21,91,45,2.02,ZA,1730351953
17,17,boli,45.7667,130.5167,13.69,33,62,1.60,CN,1730351963
18,18,puerto ayora,-0.7393,-90.3518,20.31,98,46,2.68,EC,1730351964
21,21,waitangi,-43.9535,-176.5597,12.68,82,41,2.24,NZ,1730351968
66,66,blackmans bay,-43.0167,147.3167,15.41,50,50,1.79,AU,1730352120
89,89,hamilton,39.1834,-84.5333,20.02,78,40,5.66,US,1730352146
110,110,minas de marcona,-15.2119,-75.1103,12.84,83,28,1.94,PE,1730352172
122,122,christchurch,-43.5333,172.6333,20.11,45,24,2.57,NZ,1730351710
130,130,constantia,44.1833,28.6500,11.50,66,37,2.29,RO,1730352196
131,131,breyten,-26.3018,29.9870,13.18,71,65,3.89,ZA,1730352197


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = cleaned_city_data[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = pd.NA

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
8,port alfred,ZA,-33.5906,26.8910,91,<NA>
17,boli,CN,45.7667,130.5167,33,<NA>
18,puerto ayora,EC,-0.7393,-90.3518,98,<NA>
21,waitangi,NZ,-43.9535,-176.5597,82,<NA>
66,blackmans bay,AU,-43.0167,147.3167,50,<NA>


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000
category = 'accommodation.hotel'
params = {
    'categories': category,
    'apiKey': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = hotel_df.loc[index, 'Lng']
    latitude = hotel_df.loc[index, 'Lat']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port alfred - nearest hotel: No hotel found
boli - nearest hotel: No hotel found
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
waitangi - nearest hotel: Hotel Chathams
blackmans bay - nearest hotel: Villa Howden
hamilton - nearest hotel: North Vista Manor
minas de marcona - nearest hotel: No hotel found
christchurch - nearest hotel: Ibis Hotel Christchurch
constantia - nearest hotel: Hotel Dali
breyten - nearest hotel: No hotel found
mount gambier - nearest hotel: The Old Mount Gambier Gaol
island harbour - nearest hotel: Premier Inn
lebu - nearest hotel: No hotel found
aral - nearest hotel: No hotel found
masterton - nearest hotel: Copthorne Hotel Solway Park
huarmey - nearest hotel: Hostal Santa Rosa
qamdo - nearest hotel: 银杏商务宾馆
richards bay - nearest hotel: SUN1 Richards Bay
port lincoln - nearest hotel: Boston Hotel
yafran - nearest hotel: فندق يفرن السياحي
aksu - nearest hotel: Akesu Tianyuan International Hotel
sile - nearest hotel: No 

,City,Country,Lat,Lng,Humidity,Hotel Name
8,port alfred,ZA,-33.5906,26.8910,91,No hotel found
17,boli,CN,45.7667,130.5167,33,No hotel found
18,puerto ayora,EC,-0.7393,-90.3518,98,Hostal La Mirada De Solitario George
21,waitangi,NZ,-43.9535,-176.5597,82,Hotel Chathams
66,blackmans bay,AU,-43.0167,147.3167,50,Villa Howden
89,hamilton,US,39.1834,-84.5333,78,North Vista Manor
110,minas de marcona,PE,-15.2119,-75.1103,83,No hotel found
122,christchurch,NZ,-43.5333,172.6333,45,Ibis Hotel Christchurch
130,constantia,RO,44.1833,28.6500,66,Hotel Dali
131,breyten,ZA,-26.3018,29.9870,71,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
%%capture --no-display

# Configure the map plot
map_2 = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500,
    size = 'Humidity',
    scale = .5,
    color = 'City',
    hover_cols = ['Hotel Name', 'Country']
)

# Display the map
map_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)